In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# references:

1. https://www.kaggle.com/ashwingadam/prediction-of-the-imdb-rating-using-xgboost
2. https://www.kaggle.com/srinag/movielens-rating-prediction-modeling
3. https://rpubs.com/Khonjeja/640861
4. https://rpubs.com/Jango/486734
5. https://keras.io/examples/structured_data/collaborative_filtering_movielens/
6. http://rstudio-pubs-static.s3.amazonaws.com/492432_1de5f36027184050a8d19cf674b0daeb.html
7. https://datascience-enthusiast.com/Python/cs110_lab2_als_prediction.html
8. https://paperswithcode.com/sota/collaborative-filtering-on-movielens-1m

In [13]:
# Constants
train_columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
test_columns = ['UserID', 'MovieID', 'Timestamp']
movies_columns = ['MovieID', 'Title', 'Genres']
users_columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

In [52]:
import pandas as pd 
import numpy as np 
prefix = "/content/drive/MyDrive/MovieRecommenderDataset/"
movies = pd.read_csv(prefix+'movies.txt', sep='::', header=None, engine='python')
# movies.drop(0, inplace=True, axis=1)
movies.columns = movies_columns
users = pd.read_csv(prefix+'users.txt', sep='::', header=None, engine='python')
users.columns = users_columns
training = pd.read_csv(prefix+'training.txt', sep=',', header=None, engine='python')
training.columns = train_columns
testing = pd.read_csv(prefix+'testing.txt', sep=',', header=None, engine='python')
testing.columns = test_columns

rating = pd.read_csv('ratings.dat', sep='::', header=None)
rating.columns = train_columns

# print('USER')
# users.head(3)
# print('MOVIES')
# movies.head(3)
# print('TRAIN')
training.head(3)
# print('TEST')
# testing.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,UserID,MovieID,Rating,Timestamp
0,63,3006,4,977899676
1,1125,10,2,975625890
2,181,2384,2,977088920


In [15]:
import pandas_profiling 

# pandas_profiling.ProfileReport(movies)

In [16]:
# pandas_profiling.ProfileReport(users)

In [17]:
train_ = pd.merge(training,users)
train_ = pd.merge(train_, movies)
train_.drop('Timestamp', inplace=True, axis=1)
train_.drop('Title', inplace=True, axis=1)
# train_.drop('UserID', inplace=True, axis=1)
# train_.drop('Genres', inplace=True, axis=1)
train_['Rating'] = train_['Rating'].astype(float)

train_.dropna(inplace=True)
print(train_.head(3))
display(train_.shape)

# User based
train_ub = pd.merge(training,users)
train_ub = pd.merge(train_ub, movies)
train_ub.drop('Timestamp', inplace=True, axis=1)
train_ub['Rating'] = train_ub['Rating'].astype(float)
train_ub.dropna(inplace=True)

   UserID  MovieID  Rating Gender  Age  Occupation Zip-code Genres
0      63     3006     4.0      M   18           4    54902  Drama
1    5283     3006     5.0      M   18           2    63138  Drama
2    5433     3006     3.0      F   35          17    45014  Drama


(700146, 8)

In [18]:
test_ = pd.merge(testing, users)
test_ = pd.merge(test_, movies)
test_.drop('Timestamp', inplace=True, axis=1)
test_.drop('Title', inplace=True, axis=1)
# test_.drop('UserID', inplace=True, axis=1)
print(test_.head(3))

# User based
test_ub = pd.merge(testing,users)
test_ub = pd.merge(test_ub, movies)
test_ub.drop('Timestamp', inplace=True, axis=1)
test_ub.dropna(inplace=True)


   UserID  MovieID Gender  Age  Occupation Zip-code    Genres
0     324     2803      M   35          17    55106  Thriller
1     935     2803      M   35          14    60538  Thriller
2     678     2803      M   25           0    34952  Thriller


In [19]:
test_ub['UserID'][0]

324

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_df = pd.DataFrame(
    mlb.fit_transform(train_['Genres'].str.split('|').tolist()),
    columns=mlb.classes_
    )

test_genres_df = pd.DataFrame(
    mlb.transform(test_['Genres'].str.split('|').tolist()),
    columns=mlb.classes_
    )

train_2 = train_.join(genres_df)
train_2.drop('Genres', inplace=True, axis=1)
train_2.head(3)

test_2 = test_.join(test_genres_df)
test_2.drop('Genres', inplace=True, axis=1)

In [21]:
train_2['Age'].value_counts()


25    276870
35    139318
18    128465
45     58555
50     50742
56     27155
1      19041
Name: Age, dtype: int64

In [22]:
train_2.loc[train_2['Age'] == 1].sample(5)

,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
627980,1509,2346,4.0,M,1,0,11803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
210737,4830,673,2.0,M,1,10,55317,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
462961,1468,44,1.0,M,1,14,19147,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
470154,1926,3000,5.0,F,1,0,95404,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
459136,1241,3298,3.0,M,1,10,11803,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [23]:
train_3 = train_2.copy()
test_3 = test_2.copy()
bins = [0, 9, 19, 29, 39, 49, 59, np.inf]
names = [0,1,2,3,4,5,6]
# names= ['children', 'teen', 'young adult', 'adult', 'mid-age', 'old', 'dying']
train_3['AgeRange'] = pd.to_numeric(pd.cut(train_3['Age'], bins, labels=names))
test_3['AgeRange'] = pd.to_numeric(pd.cut(test_3['Age'], bins, labels=names))
train_3.head(5)


,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,AgeRange
0,63,3006,4.0,M,18,4,54902,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,5283,3006,5.0,M,18,2,63138,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,5433,3006,3.0,F,35,17,45014,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3
3,192,3006,3.0,M,18,1,10977,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,1532,3006,5.0,M,18,14,43613,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [24]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

train_3['Gender'] = labelencoder.fit_transform(train_3['Gender'])
test_3['Gender'] = labelencoder.transform(test_3['Gender'])
train_3.head()

,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,AgeRange
0,63,3006,4.0,1,18,4,54902,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,5283,3006,5.0,1,18,2,63138,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,5433,3006,3.0,0,35,17,45014,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3
3,192,3006,3.0,1,18,1,10977,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,1532,3006,5.0,1,18,14,43613,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [25]:
train_3.groupby('AgeRange').apply(lambda s: s.sample(1))

,,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,AgeRange
AgeRange,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,164774,4006,3052,4.0,1,1,7,48038,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,60642,429,6,5.0,1,18,0,54901,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1
2,254902,575,3253,2.0,1,25,12,92130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,129545,957,3178,4.0,1,35,1,29706,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3
4,318963,3096,1372,5.0,0,45,3,38028,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4
5,601459,296,3515,1.0,1,50,5,89432,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5


In [26]:
train_3.drop('Age', inplace=True, axis=1)
test_3.drop('Age', inplace=True, axis=1)
# train_3.drop('Zip-code', inplace=True, axis=1)

In [27]:
train_final = train_3
print(train_final.head())
test_final = test_3
print(test_final.head())

   UserID  MovieID  Rating  Gender  ...  Thriller War  Western  AgeRange
0      63     3006     4.0       1  ...         0   0        0         1
1    5283     3006     5.0       1  ...         0   0        0         1
2    5433     3006     3.0       0  ...         0   0        0         3
3     192     3006     3.0       1  ...         0   0        0         1
4    1532     3006     5.0       1  ...         0   0        0         1

[5 rows x 25 columns]
   UserID  MovieID  Gender  Occupation  ... Thriller  War  Western  AgeRange
0     324     2803       1          17  ...        1    0        0         3
1     935     2803       1          14  ...        1    0        0         3
2     678     2803       1           0  ...        1    0        0         2
3    2887     2803       1           3  ...        1    0        0         2
4    3720     2803       1           7  ...        1    0        0         4

[5 rows x 24 columns]


In [28]:
# columns = [ "Zip-code", "UserID", "MovieID", "Rating"]

# columns = [ "Zip-code", "Occupation", "Gender", "AgeRange"]

columns = [ "Zip-code", "Occupation", "Gender", "AgeRange"]


train_labels = train_final["Rating"]
# train_3.drop('Age', inplace=True, axis=1)
train_data = train_final.drop(columns=columns+["Rating"])
test_data = test_final.drop(columns=columns)
train_data

# train_labels = final_df_["Rating"]
# # train_3.drop('Age', inplace=True, axis=1)
# train_data = final_df_.drop(columns=["Rating"])
# test_data = test_data_
# train_data

final_df = train_data.join(train_labels)
final_df_ = final_df.groupby(["UserID", "MovieID"])[final_df.columns.values[2:]].sum().reset_index()
test_data_ = test_data.groupby(["UserID", "MovieID"])[test_data.columns.values[2:]].sum().reset_index()
display(final_df_.head())
display(test_data_.head())

,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Rating
0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0
1,1,260,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,4.0
2,1,588,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
3,1,594,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
4,1,595,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5.0


,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,48,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,1,150,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1,527,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,531,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,919,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


# New Section

In [ ]:
# from catboost import CatBoostRegressor

# model = CatBoostRegressor(iterations=100, learning_rate=1, depth=2, verbose=False)
# # Fit model
# model.fit(train_data, train_labels)
# # Get predictions
# preds = model.predict(test_data)

In [ ]:
# preds[:1000]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.33, random_state=7)


# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

random_forest = RandomForestRegressor(max_depth=10, random_state=0)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
print(Y_pred)
print(y_test)
# acc_random_forest = round(random_forest.score(y_test, Y_pred) * 100, 2)
# print(acc_random_forest)
mse = mean_squared_error(y_test, Y_pred)
print("rmse: %.2f" % (mse**0.5))

# New Section

# New Section

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(max_depth=10, random_state=0)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
print(Y_pred)
print(y_test)
# acc_random_forest = round(random_forest.score(y_test, Y_pred) * 100, 2)
# print(acc_random_forest)
mse = mean_squared_error(y_test, Y_pred)
print("rmse: %.2f" % (mse**0.5))

In [ ]:
random_forest = RandomForestRegressor(max_depth=10, random_state=0)
random_forest.fit(train_data, train_labels)
pred = random_forest.predict(test_data)

In [ ]:
print(pred[:1000])

In [ ]:
print("rmse: %.2f" % mse)

In [ ]:
Y_pred[:9]

# XGBoost Regressor

In [ ]:
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

xgb1 = XGBRegressor(n_estimators=100, learning_rate=0.2, gamma=0, subsample=0.75,
colsample_bytree=1, max_depth=7)
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

# params['eval_metric'] = "rmse"
# num_boost_round = 999

# xgb_grid = GridSearchCV(xgb1,
#                         parameters,
#                         scoring='neg_root_mean_squared_error',
#                         cv = 2,
#                         n_jobs = 5,
#                         verbose=True)


# xgb_grid.fit(train_data,
#          train_labels)

# xgb1.fit(train_data,
#           train_labels)

xgb1.fit(X_train,
          y_train)

# print(xgb1.best_score_)
# print(xgb1.best_params_)

# model.fit(X_train, y_train)
# make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# mse = mean_squared_error(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# print("rmse: %.2f%%" % (mse**0.5))



In [ ]:
preds = xgb1.predict(X_test)
print("rmse: %.2f" % (mse**0.5))
print(preds[1000:2000])
print(y_test)
mse = mean_squared_error(preds,y_test)


In [ ]:
from xgboost import XGBClassifier
xgb2 = XGBClassifier(n_estimators=100, learning_rate=0.05, gamma=0, subsample=0.75,
colsample_bytree=1, max_depth=7)

xgb2.fit(X_train,
          y_train)

preds2 = xgb2.predict(X_test)
print(preds2[5])
print(y_test)
mse = mean_squared_error(preds2,y_test)
print("rmse: %.2f" % (mse**0.5))

In [ ]:
print(preds2[:5])
print(y_test)

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
print(y_test.value_counts())

# Collaborative Filtering with Keras

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [ ]:
train_data.head()

In [ ]:
df = train_data.copy()
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["MovieID"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Movie"] = df["MovieID"].map(movie2movie_encoded)
num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)

In [ ]:
df.head()

In [ ]:
# df = train_data.sample(frac=1, random_state=42)
# df = train_data.join(train_labels)
df = df.join(train_labels)
df = df.drop(columns=['UserID', 'MovieID'])
# x = df[["User", "Movie"]].values
x = df.iloc[:,:-1].values
min_rating = min(df["Rating"])
max_rating = max(df["Rating"])
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df["Rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [ ]:
print(x_train[0:3])

In [ ]:
df.head()

In [ ]:
EMBEDDING_SIZE = 50


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

predict

In [ ]:
pred_nn = model.predict(x_val).flatten()

In [ ]:
for i,rating in enumerate(pred_nn):
    pred_nn[i]= pred_nn[i] * (max_rating - min_rating) + min_rating
print(pred_nn)

mse = mean_squared_error(pred_nn,y_val)
print("rmse: %.2f" % (mse**0.5))


# Bayesian FM

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
!pip install myfm
import myfm


     |████████████████████████████████| 92kB 5.4MB/s 
  Using cached https://files.pythonhosted.org/packages/00/84/fc9dc13ee536ba5e6b8fd10ce368fea5b738fe394c3b296cde7c9b144a92/pybind11-2.6.1-py2.py3-none-any.whl
  Created wheel for myfm: filename=myfm-0.3.0.0-cp36-cp36m-linux_x86_64.whl size=4881988 sha256=cb810386993852db0c24d26b999f152b37f2a008845ffa3b08601dd3bc78214e
  Stored in directory: /root/.cache/pip/wheels/b0/65/85/56f057f67b4b59012f92e933c5a331f752b5355fefc194dae8
Successfully built myfm


In [ ]:
train_data.head()

,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,63,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,5283,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,5433,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,192,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1532,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df_train = X_train.join(y_train)
# df_test = X_test.join(y_test)

def test_myfm(df_train, df_test, rank=8, grouping=None, n_iter=100, samples=95, final_predict=False):
    # explanation_columns = ['UserID', 'MovieID']
    ohe = OneHotEncoder(handle_unknown='ignore')
    X_train = ohe.fit_transform(df_train.iloc[:,:-1])
    if(not final_predict):
      X_test = ohe.transform(df_test.iloc[:,:-1])
    else:
      X_test = ohe.transform(df_test)
    # X_train = ohe.fit_transform(df_train[explanation_columns])
    # X_test = ohe.transform(df_test[explanation_columns])
    y_train = df_train.Rating.values

    # print(X_train)
    fm = myfm.MyFMRegressor(rank=rank, random_seed=114514)

    if grouping:
        # assign group index for each column of X_train.
        grouping = [ i for i, category in enumerate(ohe.categories_) for _ in category]
        assert len(grouping) == X_train.shape[1]

    fm.fit(X_train, y_train, grouping=grouping, n_iter=n_iter, n_kept_samples=samples)
    prediction = fm.predict(X_test)

    if(not final_predict):
      y_test = df_test.Rating.values
      rmse = ((y_test - prediction) ** 2).mean() ** .5
      mae = np.abs(y_test - prediction).mean()
      print('rmse={rmse}, mae={mae}'.format(rmse=rmse, mae=mae))
    return fm, ohe, prediction


# fm, preds = test_myfm(final_df, df_test, rank=8)


In [ ]:
df_train = X_train.join(y_train)
df_train_ = df_train.groupby(["UserID", "MovieID"])[df_train.columns.values[2:]].sum().reset_index()
df_test = X_test.join(y_test)
df_test_ = df_test.groupby(["UserID", "MovieID"])[df_test.columns.values[2:]].sum().reset_index()
display(df_train_.head())
fm, ohe, preds = test_myfm(df_train_, df_test_, rank=2, final_predict=False)

# rank2 rmse 0.8792
# grouping = 0.8814
# 0.877 after groupby manual?

,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Rating
0,1,588,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
1,1,608,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,4.0
2,1,720,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0
3,1,783,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
4,1,914,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,3.0


alpha = 1.37 w0 = 3.60 : 100%|██████████| 100/100 [00:27<00:00,  3.64it/s]


rmse=0.8770311786649296, mae=0.6914848741986341


In [ ]:
final_df = train_data.join(train_labels)
final_df_ = final_df.groupby(["UserID", "MovieID"])[final_df.columns.values[2:]].sum().reset_index()
test_data_ = test_data.groupby(["UserID", "MovieID"])[test_data.columns.values[2:]].sum().reset_index()
display(final_df_.head())
display(test_data_.head())

,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Rating
0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0
1,1,260,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,4.0
2,1,588,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
3,1,594,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0
4,1,595,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5.0


,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,48,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,1,150,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1,527,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,531,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,919,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [ ]:
fm, ohe, preds = test_myfm(final_df_, test_data_, rank=2, final_predict=True)

alpha = 1.38 w0 = 3.29 : 100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


In [ ]:
print(len(preds))
print(len(test_data))
print(preds[:100])

300063
300063
[3.27067637 4.37316941 4.71276484 3.93135748 4.44668921 3.78037041
 3.90032606 4.26627618 4.43000207 3.93386982 4.04452682 3.99756878
 3.87455734 4.24932144 3.97774573 4.29545315 3.39054223 3.22302568
 2.96518663 3.55401256 2.89632925 2.90525428 3.01599175 3.38597435
 3.947199   3.89038491 3.69497629 4.20673824 3.84751069 4.05454245
 3.46043932 3.52142516 3.93324756 3.68498682 4.04812228 3.75031511
 3.46378446 3.88861019 3.26616752 4.00989115 3.20363083 3.91341208
 3.96297918 4.21764886 3.90745053 3.77953309 3.58872038 3.69263977
 3.35228216 3.97060625 3.82551858 3.08512371 4.06762506 3.8434272
 3.43804497 4.05198073 3.66570413 4.40084257 4.17302763 4.33306214
 4.33772376 3.39523465 3.60142319 3.92314587 3.83267905 3.88256062
 3.31384402 3.64291181 4.00616127 3.38842023 4.47679397 4.08412254
 4.38064458 4.1125065  4.54993207 4.16527619 3.52834445 3.12373366
 3.05988149 4.2911092  3.58064063 2.94282704 3.16476477 3.41981506
 3.23877872 3.29187914 3.57012493 4.33655298 3.75

# Factorization Machine

In [ ]:
#One hot encoding
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, normalize
def OneHotEncoding(train,test):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(train)
    train = encoder.transform(train)
    test = encoder.transform(test)
    return train, test

In [ ]:
# !pip install fastFM
from fastFM import als

# xtrain, xtest, ytrain, ytest = split_testtrain(ratings, 0.2)
# xtrain_enc, xtest_enc = OneHotEncoding(xtrain, xtest)

#Factorization machine
fm = als.FMRegression(n_iter=100, rank=2, l2_reg_w=0.1, l2_reg_V=0.1)
xtrain_enc, xtest_enc = OneHotEncoding(final_df_.iloc[:,:-1], test_data_)

fm.fit(xtrain_enc, final_df_.iloc[:,-1])
predictions = fm.predict(xtest_enc)

#Evaluation metrics
# rmse = sqrt(mean_squared_error(ytest,predictions))
# mae = mean_absolute_error(ytest,predictions)
# coverage = rec_coverage(xtest, ytest, predictions, 10)
# print("rmse", rmse)


In [ ]:
predictions[:100]

array([3.38355116, 4.46458789, 4.77861055, 4.00740733, 4.50623167,
       3.8334357 , 3.94786304, 4.38030973, 4.39593153, 3.84682394,
       4.16027485, 4.09975531, 3.91013915, 4.35466737, 4.08965862,
       4.21784687, 3.23313407, 2.54600751, 2.19551555, 3.73144143,
       2.77408463, 2.73281289, 2.74124831, 3.32455196, 4.30692185,
       4.24731511, 4.08411636, 4.26940809, 3.87195759, 4.13179946,
       3.05641697, 2.9759393 , 3.96352447, 3.61144555, 4.15454019,
       3.82692794, 3.47448237, 4.01906257, 3.12741348, 4.18110685,
       3.18722381, 4.01986453, 4.29408402, 4.10886766, 3.86278594,
       4.28287215, 3.61973129, 3.81894253, 3.42656623, 4.12592668,
       4.00338931, 3.41768863, 3.89423424, 3.90911056, 3.68509275,
       4.32405063, 3.88243643, 4.34089701, 3.98424113, 4.28161461,
       4.50874258, 3.53441717, 3.26358918, 4.04718876, 3.8367971 ,
       4.09754777, 3.4968134 , 2.9937551 , 4.01875025, 3.45230537,
       4.50054628, 4.00751614, 4.44669468, 4.11101065, 4.68889

# SVD

In [7]:
import pandas as pd
import numpy as np
!pip install surprise
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate

from surprise import Reader

     |████████████████████████████████| 11.8MB 4.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670968 sha256=5cfb7891afd4793e250ce347df27a6a1e3c21b2063c2461ce835418058bd46ef
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
final_df_.head()

,UserID,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Rating
0,63,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4.0
1,5283,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5.0
2,5433,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3.0
3,192,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3.0
4,1532,3006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5.0


In [29]:
reader = Reader()
data = Dataset.load_from_df(final_df_[['UserID', 'MovieID', 'Rating']], reader)

In [30]:
from surprise.model_selection import GridSearchCV

svd = SVD()

# optimal params from https://github.com/microsoft/recommenders/blob/84444c5f40237bea69e44f7b2fb5097530ed9920/notebooks/04_model_select_and_optimize/azureml_hyperdrive_tuning_svd.ipynb

# --datastore $AZUREML_DATAREFERENCE_workspaceblobstore 
# --train-datapath data/movielens_100k_train.pkl 
# --validation-datapath data/movielens_100k_val.pkl 
# --output_dir ./outputs --surprise-reader ml-100k 
# --rating-metrics rmse --ranking-metrics precision_at_k ndcg_at_k 
# --usercol userID --itemcol itemID --k 10 --random-state 0 
# --epochs 30 --biased --verbose --n_factors 100 
# --init_mean -0.00373842687107999 --init_std_dev 0.0102254534112653 
# --lr_bu 1E-06 --lr_bi 1E-06 --lr_pu 1E-06 --lr_qi 0.00611767666611114 
# --reg_bu 0.920265632488417 --reg_bi 0.515315667765222 
# --reg_pu 0.258131187700377 --reg_qi 1E-06

param_grid = { 'n_epochs': [50,100,200,250]
            }

# best params?
# 'n_epochs': [100]
# n_factors: 150
# lr_all: 0.005
# reg_all: 0.02

# try:
# reg_all 0.015?
# init_stdev=0.1
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

KeyboardInterrupt: ignored

In [31]:
svd_final = SVD(
    n_factors= 100,
    n_epochs= 200, 
    lr_all= 0.005,  # or 0.01?
    reg_all= 0.02)

trainset = data.build_full_trainset()
svd_final.fit(trainset)


In [32]:
predictions = []
for index, row in testing.iterrows():
    # print(row)
    predictions.append(svd_final.predict(row['UserID'], row['MovieID']).est)
    # break
predictions

[3.7736285745274216,
 2.803346512350932,
 3.48175692530211,
 3.7425982301151146,
 4.626636979764742,
 3.3507367369410153,
 3.5802324731125723,
 3.1237190607843295,
 3.46832421978412,
 2.808740539606293,
 3.897415242156248,
 3.5308224901042102,
 3.567673987382939,
 5,
 3.534757807041744,
 3.901934534458646,
 3.6942748941642476,
 2.8388892572896225,
 4.044029144413425,
 3.2989722131279158,
 3.8799517796252845,
 4.590572173636387,
 3.675896932667797,
 3.268569369246497,
 2.478175689039106,
 4.663048082895202,
 2.819628216639774,
 3.3373053936337738,
 4.202095506584332,
 4.13258197673259,
 2.9566970103200334,
 4.361904429676159,
 4.170842480626345,
 3.321881832300967,
 2.814053940068231,
 4.4644027346392665,
 3.1392609672710705,
 3.824555525658225,
 4.7053781573969475,
 4.7429379072843805,
 4.943442874987028,
 3.2327173304396277,
 4.039234377121685,
 4.968071851078094,
 4.1846297433572355,
 3.6064902510048604,
 2.975631639222573,
 3.435166818304423,
 3.629387786953555,
 4.141793676530607,


# Prepare submission

In [33]:
import pandas as pd 

df = pd.DataFrame(predictions, columns=['Predicted'])
df.index = df.index + 1
df.index.names = ['ID']
df.head()
df.to_csv('SVDTuned5.csv')

In [38]:
merged = pd.merge(testing, rating, how='inner', on=['UserID', 'MovieID'])

In [59]:
merged.head(10)

,UserID,MovieID,Timestamp_x,Rating,Timestamp_y
0,324,2803,976406738,4,976406738
1,3167,3253,968818650,4,968818650
2,1209,1459,974842767,2,974842767
3,2298,3869,974500026,3,974500026
4,1694,916,974715889,4,974715889
5,890,2424,980132318,2,980132318
6,343,435,988597018,4,988597018
7,2288,235,974520309,4,974520309
8,935,480,975181384,3,975181384
9,850,2132,975351212,5,975351212


In [53]:
rating.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [55]:
rating['UserID'].

0             1
1             1
2             1
3             1
4             1
           ... 
1000204    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: UserID, Length: 1000209, dtype: int64

In [57]:
merged.head(100)

,UserID,MovieID,Timestamp_x,Rating,Timestamp_y
0,324,2803,976406738,4,976406738
1,3167,3253,968818650,4,968818650
2,1209,1459,974842767,2,974842767
3,2298,3869,974500026,3,974500026
4,1694,916,974715889,4,974715889
...,...,...,...,...,...
95,1312,1025,974780592,5,974780592
96,1314,1487,975533351,4,975533351
97,2793,2161,972935183,2,972935183
98,1110,2690,974920382,3,974920382


In [42]:
print(len(merged))
print(len(testing))
print(len(rating))

154461
300063
514876
